## TCNAE

In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-10-16 13:11:15.550334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-16 13:11:15.550373: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [4]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter3/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter3/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


In [5]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/TCN_HYP.csv')

input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
# source_dict_results_train =  {}
# source_dict_results_test =  {}

# for file in total_list:
#     df= df_dict_total[file]
#     xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
#     xtrain = temporalize(xtrain)
#     xtest = temporalize(xtest)
#     ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
#     ytest = ytest.reset_index()
#     ytest = ytest['anomaly']
#     model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
#     model  = TCNAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
#     model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
#     test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
#     lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
#     source_dict_results_test[file] = test_df
#     source_dict_results_train[file] = train_df
# with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_TcnAE.pkl', 'wb') as f:
#     pickle.dump(source_dict_results_test, f)
# with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_TcnAE.pkl', 'wb') as f:
#     pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = temporalize(train_merge_df)

model = TCNAEModel(layer1=30, layer2=20, encoding_dim=6)
model.fit(train_merge_df, train_merge_df, epochs=100, batch_size=256, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_TcnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_TcnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

2023-08-24 16:32:30.957331: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-24 16:32:30.957467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-08-24 16:32:30.957557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-08-24 16:32:30.957649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-08-24 16:32:30.957737: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Epoch 1/100
55/55 - 10s - loss: 0.0562 - val_loss: 0.0563 - 10s/epoch - 180ms/step
Epoch 2/100
55/55 - 5s - loss: 0.0255 - val_loss: 0.0505 - 5s/epoch - 88ms/step
Epoch 3/100
55/55 - 5s - loss: 0.0213 - val_loss: 0.0478 - 5s/epoch - 87ms/step
Epoch 4/100
55/55 - 5s - loss: 0.0191 - val_loss: 0.0443 - 5s/epoch - 87ms/step
Epoch 5/100
55/55 - 5s - loss: 0.0177 - val_loss: 0.0432 - 5s/epoch - 87ms/step
Epoch 6/100
55/55 - 5s - loss: 0.0171 - val_loss: 0.0429 - 5s/epoch - 87ms/step
Epoch 7/100
55/55 - 5s - loss: 0.0167 - val_loss: 0.0423 - 5s/epoch - 87ms/step
Epoch 8/100
55/55 - 5s - loss: 0.0165 - val_loss: 0.0425 - 5s/epoch - 87ms/step
Epoch 9/100
55/55 - 5s - loss: 0.0163 - val_loss: 0.0419 - 5s/epoch - 88ms/step
Epoch 10/100
55/55 - 5s - loss: 0.0164 - val_loss: 0.0418 - 5s/epoch - 87ms/step
Epoch 11/100
55/55 - 5s - loss: 0.0159 - val_loss: 0.0412 - 5s/epoch - 87ms/step
Epoch 12/100
55/55 - 5s - loss: 0.0159 - val_loss: 0.0407 - 5s/epoch - 87ms/step
Epoch 13/100
55/55 - 5s - loss: 0.

## TCNATTNAE

In [2]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-09-08 18:44:33.364244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-08 18:44:33.364331: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [7]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter2/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter2/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


In [7]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/TCNATTN_HYP.csv')

input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_train =  {}
source_dict_results_test =  {}

for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = TCNAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_TcnAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_TcnAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = temporalize(train_merge_df)

model = TCNAttnAEModel(layer1=30, layer2=30, encoding_dim=6)
model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=256, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_TcnAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_TcnAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

Epoch 1/200
70/70 - 10s - loss: 0.0292 - val_loss: 0.0097 - 10s/epoch - 141ms/step
Epoch 2/200
70/70 - 4s - loss: 0.0070 - val_loss: 0.0068 - 4s/epoch - 62ms/step
Epoch 3/200
70/70 - 5s - loss: 0.0062 - val_loss: 0.0059 - 5s/epoch - 65ms/step
Epoch 4/200
70/70 - 5s - loss: 0.0058 - val_loss: 0.0059 - 5s/epoch - 65ms/step
Epoch 5/200
70/70 - 4s - loss: 0.0055 - val_loss: 0.0058 - 4s/epoch - 64ms/step
Epoch 6/200
70/70 - 4s - loss: 0.0054 - val_loss: 0.0058 - 4s/epoch - 61ms/step
Epoch 7/200
70/70 - 4s - loss: 0.0053 - val_loss: 0.0057 - 4s/epoch - 63ms/step
Epoch 8/200
70/70 - 4s - loss: 0.0053 - val_loss: 0.0059 - 4s/epoch - 61ms/step
Epoch 9/200
70/70 - 4s - loss: 0.0052 - val_loss: 0.0060 - 4s/epoch - 63ms/step
Epoch 10/200
70/70 - 5s - loss: 0.0051 - val_loss: 0.0061 - 5s/epoch - 66ms/step
Epoch 11/200
70/70 - 4s - loss: 0.0051 - val_loss: 0.0061 - 4s/epoch - 61ms/step
Epoch 12/200
70/70 - 4s - loss: 0.0050 - val_loss: 0.0062 - 4s/epoch - 61ms/step
Epoch 13/200
70/70 - 4s - loss: 0.

70/70 - 7s - loss: 0.0037 - val_loss: 0.0047 - 7s/epoch - 100ms/step
Epoch 103/200
70/70 - 7s - loss: 0.0038 - val_loss: 0.0048 - 7s/epoch - 101ms/step
Epoch 104/200
70/70 - 7s - loss: 0.0037 - val_loss: 0.0052 - 7s/epoch - 101ms/step
Epoch 105/200
70/70 - 7s - loss: 0.0037 - val_loss: 0.0048 - 7s/epoch - 99ms/step
Epoch 106/200
70/70 - 6s - loss: 0.0037 - val_loss: 0.0046 - 6s/epoch - 80ms/step
Epoch 107/200
70/70 - 5s - loss: 0.0036 - val_loss: 0.0048 - 5s/epoch - 78ms/step
Epoch 108/200
70/70 - 5s - loss: 0.0037 - val_loss: 0.0048 - 5s/epoch - 78ms/step
Epoch 109/200
70/70 - 6s - loss: 0.0036 - val_loss: 0.0049 - 6s/epoch - 80ms/step
Epoch 110/200
70/70 - 6s - loss: 0.0037 - val_loss: 0.0051 - 6s/epoch - 79ms/step
Epoch 111/200
70/70 - 6s - loss: 0.0037 - val_loss: 0.0048 - 6s/epoch - 80ms/step
Epoch 112/200
70/70 - 6s - loss: 0.0037 - val_loss: 0.0049 - 6s/epoch - 80ms/step
Epoch 113/200
70/70 - 6s - loss: 0.0036 - val_loss: 0.0054 - 6s/epoch - 84ms/step
Epoch 114/200
70/70 - 6s - 

Epoch 2/200
70/70 - 6s - loss: 0.0073 - val_loss: 0.0073 - 6s/epoch - 92ms/step
Epoch 3/200
70/70 - 7s - loss: 0.0057 - val_loss: 0.0064 - 7s/epoch - 93ms/step
Epoch 4/200
70/70 - 7s - loss: 0.0054 - val_loss: 0.0057 - 7s/epoch - 99ms/step
Epoch 5/200
70/70 - 7s - loss: 0.0051 - val_loss: 0.0054 - 7s/epoch - 99ms/step
Epoch 6/200
70/70 - 7s - loss: 0.0048 - val_loss: 0.0054 - 7s/epoch - 100ms/step
Epoch 7/200
70/70 - 6s - loss: 0.0045 - val_loss: 0.0048 - 6s/epoch - 91ms/step
Epoch 8/200
70/70 - 7s - loss: 0.0043 - val_loss: 0.0048 - 7s/epoch - 94ms/step
Epoch 9/200
70/70 - 7s - loss: 0.0042 - val_loss: 0.0046 - 7s/epoch - 96ms/step
Epoch 10/200
70/70 - 7s - loss: 0.0041 - val_loss: 0.0048 - 7s/epoch - 104ms/step
Epoch 11/200
70/70 - 7s - loss: 0.0039 - val_loss: 0.0051 - 7s/epoch - 99ms/step
Epoch 12/200
70/70 - 7s - loss: 0.0038 - val_loss: 0.0055 - 7s/epoch - 104ms/step
Epoch 13/200
70/70 - 7s - loss: 0.0038 - val_loss: 0.0047 - 7s/epoch - 102ms/step
Epoch 14/200
70/70 - 7s - loss: 

Epoch 103/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0044 - 7s/epoch - 99ms/step
Epoch 104/200
70/70 - 7s - loss: 0.0030 - val_loss: 0.0042 - 7s/epoch - 97ms/step
Epoch 105/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0046 - 7s/epoch - 94ms/step
Epoch 106/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0045 - 7s/epoch - 97ms/step
Epoch 107/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0042 - 7s/epoch - 94ms/step
Epoch 108/200
70/70 - 6s - loss: 0.0029 - val_loss: 0.0047 - 6s/epoch - 93ms/step
Epoch 109/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0044 - 7s/epoch - 97ms/step
Epoch 110/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0042 - 7s/epoch - 98ms/step
Epoch 111/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0044 - 7s/epoch - 95ms/step
Epoch 112/200
70/70 - 7s - loss: 0.0029 - val_loss: 0.0045 - 7s/epoch - 98ms/step
Epoch 113/200
70/70 - 6s - loss: 0.0029 - val_loss: 0.0043 - 6s/epoch - 85ms/step
Epoch 114/200
70/70 - 6s - loss: 0.0028 - val_loss: 0.0045 - 6s/epoch - 81ms/step
Epoch 115/200
70

KeyboardInterrupt: 